In [1]:
import platform
import time
import torch
import torch.cuda
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder


In [2]:
CUDA_ENABLED = torch.cuda.is_available()
if CUDA_ENABLED:
    print(torch.cuda.get_device_name())
else:
    print(platform.processor())
device = torch.device('cuda' if CUDA_ENABLED else 'cpu')

NVIDIA GeForce RTX 4090


In [3]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, (5, 3), padding=(3, 1), stride=(3, 1), dilation=(2, 1))
        self.conv2 = nn.Conv2d(64, 128, (5, 3), padding=(3, 1), stride=(1, 1), dilation=(1, 1))
        self.conv3 = nn.Conv2d(128, 256, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))
        self.bn1 = nn.BatchNorm2d(64, affine=True)
        self.bn2 = nn.BatchNorm2d(128, affine=True)
        self.bn3 = nn.BatchNorm2d(256, affine=True)
        self.pool = nn.MaxPool2d((2, 1))
        self.fc = nn.Linear(46080, 4)
        

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
BATCH_SIZE = 32
EPOCH = 100
LEARNING_RATE = 0.01

def train_model(dataset_folder: str):
    cnn_model = CNNet().to(device)
    
    #optimizer 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(cnn_model.parameters(), lr=LEARNING_RATE)
    
    # data source
    transform = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()])
    dataset = ImageFolder(dataset_folder, transform=transform)
    train_data, test_data = random_split(dataset, [0.7, 0.3])
    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
    
    # training
    time_now = int(time.time())
    for epoch in range(EPOCH):
        total_loss = 0.0
        correct = 0
        total = 0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = cnn_model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs.data, 1)
            total_loss += loss.item()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'[Epoch {epoch + 1}] - Avg Loss = {total_loss / len(train_loader)} - Correct = {correct} - Accuracy = {correct / total}')
              
        torch.save({
            'epoch': epoch,
            'model_state_dict': cnn_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }, f'./{time_now}.pth')
            
            
            
train_model('../../../dataset/ohlc_graphs/nikkei_mid_small_cap/20d')

[Epoch 1] - Avg Loss = 1.408521329543012 - Correct = 175114 - Accuracy = 0.3346724256555309
[Epoch 2] - Avg Loss = 1.4083400872151446 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 3] - Avg Loss = 1.4083171539198163 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 4] - Avg Loss = 1.4083228875206757 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 5] - Avg Loss = 1.4083343529946184 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 6] - Avg Loss = 1.4083343533955792 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 7] - Avg Loss = 1.408328620334194 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 8] - Avg Loss = 1.4083286201300687 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 9] - Avg Loss = 1.4083286200936176 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 10] - Avg Loss = 1.4083343529508772 - Correct = 175462 - Accuracy = 0.3353375124225977
[Epoch 11] - Avg Loss = 1.4083228849763973 - Correct = 175462 - Accurac